In [1]:
# ✅ Install dependencies
!pip install -q diffusers transformers accelerate safetensors gradio

import torch, gc
import gradio as gr
from diffusers import StableDiffusionXLPipeline

# ✅ Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Use a photorealistic SDXL model
model_id = "SG161222/RealVisXL_V4.0"

# ✅ Load pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    use_safetensors=True,
    variant="fp16" if device=="cuda" else None
).to(device)

# ✅ Memory optimization
pipe.enable_attention_slicing()

# ✅ Expand prompt → India-rooted + special-case for "big cat"
def expand_prompt(user_prompt: str) -> str:
    if user_prompt.strip().lower() == "big cat":
        # Special handling: oversized domestic cat
        return (
            "high quality realistic photograph of a giant oversized domestic house cat, "
            "cute, fluffy, extremely large, sitting naturally in an Indian street, "
            "present-day India, authentic Indian environment, "
            "natural lighting, vibrant colors"
        )
    else:
        # General India-rooted prompt
        return (
            f"high quality realistic photograph of {user_prompt}, "
            f"present-day India, authentic Indian environment, "
            f"natural lighting, vibrant colors, "
            f"must include exactly: {user_prompt}"
        )

# ✅ Generate image with GPU memory cleanup
def generate_image(user_prompt):
    prompt = expand_prompt(user_prompt)

    result = pipe(
        prompt=prompt,
        negative_prompt="cartoon, anime, western style, unrealistic, fake, painting, illustration",
        height=704,    # good balance HQ + GPU safety
        width=704,
        guidance_scale=8,
        num_inference_steps=35
    )
    image = result.images[0]

    # ✅ Free GPU/CPU memory to avoid crash
    if hasattr(pipe, "_last_latents"):
        del pipe._last_latents
    torch.cuda.empty_cache()
    gc.collect()

    return image

# ✅ Gradio Interface
demo = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(
        label="Enter anything (India-rooted)",
        placeholder="Example: busy Indian street in Delhi, Ganesh Chaturthi festival, Arabian Sea in Mumbai"
    ),
    outputs="image",
    title="Text to Image (Indian Version)",
    description="Type anything (buildings, rivers, festivals, offices, traffic, people, landscapes, etc.) → generates realistic, present-day India-rooted images. Special fix: 'big cat' = oversized domestic cat."
)

demo.launch(debug=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/5.14G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f746dab4d407592fcb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
